In [14]:

import tensorflow as tf


import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [15]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []


for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)



training_sentences[0]


training_labels[0]


vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [17]:
num_epochs = 10
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 279us/sample - loss: 0.4851 - accuracy: 0.7530 - val_loss: 0.3460 - val_accuracy: 0.8496
Epoch 2/10
25000/25000 [==============================] - 6s 260us/sample - loss: 0.2395 - accuracy: 0.9073 - val_loss: 0.3701 - val_accuracy: 0.8394
Epoch 3/10
25000/25000 [==============================] - 6s 255us/sample - loss: 0.0928 - accuracy: 0.9754 - val_loss: 0.4493 - val_accuracy: 0.8282
Epoch 4/10
25000/25000 [==============================] - 6s 249us/sample - loss: 0.0221 - accuracy: 0.9974 - val_loss: 0.5352 - val_accuracy: 0.8260
Epoch 5/10
25000/25000 [==============================] - 6s 259us/sample - loss: 0.0053 - accuracy: 0.9998 - val_loss: 0.6013 - val_accuracy: 0.8250
Epoch 6/10
25000/25000 [==============================] - 6s 252us/sample - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6519 - val_accuracy: 0.8269
Epoch 7/10
25000/25000 [==========================

In [18]:
score = model.evaluate(testing_padded, testing_labels_final)

25000/25000 [==============================] - 2s 77us/sample - loss: 0.8035 - accuracy: 0.8286


In [19]:
print('Test accuracy:', score[1])

Test accuracy: 0.82864
